This file is to test my model

In [1]:
run_python_script = False

In [2]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT

In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from easydict import EasyDict as edict
import time

if not run_python_script:
    plt.style.use(FIG_ROOT/"base.mplstyle")
    %matplotlib inline

In [4]:
import importlib
import models.main_model
importlib.reload(models.main_model)

<module 'models.main_model' from '/data/rajlab1/user_data/jin/MyResearch/gTVDN-NN/notebooks/../mypkg/models/main_model.py'>

In [5]:
from models.main_model import myNet
from utils.misc import delta_time, load_pkl_folder2dict, save_pkl_dict2folder

In [6]:
# pkgs for pytorch (on Apr 3, 2023)
import torch
import torch.nn as nn
from torch.functional import F
from torch.optim.lr_scheduler import ExponentialLR

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    torch.cuda.set_device(2)
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda")
else:
    torch.set_default_tensor_type(torch.DoubleTensor)
    device = torch.device("cpu")

# Load data

In [7]:
AD_dir = DATA_ROOT/"AD_data"
ctrl_p1 = list(AD_dir.glob("70*.mat"))[0]
ctrl_p2 = list(AD_dir.glob("time*.mat"))[0]

In [8]:
from scipy.io import loadmat
import mat73

data_p1 = loadmat(ctrl_p1)["dk10"]
data_p2 = mat73.loadmat(ctrl_p2)["dk10"]
data = np.concatenate([data_p1,  data_p2], axis=0)

In [9]:
from scipy.signal import detrend
# detrend the data
data = detrend(data)

In [10]:
test_data = data[-22:].transpose(0, 2, 1)
train_data = data[:-22].transpose(0, 2, 1)

# training

In [11]:
def get_batch(dataset, batch_size, block_size):
    sub_ix = torch.randint(len(dataset), (1, ))
    ix = torch.randint(len(dataset[sub_ix]) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((dataset[sub_ix, i:i+block_size])) for i in ix])
    y = torch.stack([torch.from_numpy((dataset[sub_ix, i+1:i+1+block_size])) for i in ix])
    if device.type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x.double(), y.double()

In [12]:
def generate_position_encode(block_size, nfeature):
    # create a matrix with shape (blocksize, nfeature)
    position = torch.arange(block_size, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, nfeature, 2).float() * (-np.log(10000.0) / nfeature))
    # apply sine to even indices in the array
    pos_enc = torch.zeros((block_size, nfeature))
    pos_enc[:, 0::2] = torch.sin(position * div_term)
    # apply cosine to odd indices in the array
    pos_enc[:, 1::2] = torch.cos(position * div_term)
    return pos_enc

In [16]:
def out2pred(net_out, X):
    A_mats = net_out.reshape(config.batch_size, config.block_size, paras.nroi, -1)
    Y_pred = (A_mats @ X.unsqueeze(-1)).squeeze()
    return Y_pred

def evaluate(net):
    X_test, Y_test = get_batch(test_data, 
                     batch_size=config.batch_size,  
                     block_size=config.block_size)
    X_test = X_test + pos_enc
    net.eval()
    net_out = net(X_test)
    Y_pred = out2pred(net_out, X_test)
    loss = loss_fn(Y_test, Y_pred)
    net.train()
    return loss.item()

In [20]:
paras = edict()
paras.nroi = data.shape[1]

config = edict()
config.nfeature = paras.nroi # the dim of features at each time point
config.ndim = 256 # the output of the first FC layer
config.target_dim = paras.nroi * paras.nroi # the target dim 
config.dropout = 0.5 # the dropout rate
config.n_layer = 1 # the number of self-attention layers
config.n_head = 8 # numher of heads for multi-head attention
config.is_mask = True # Use mask to make the attention causal
config.is_bias = True # Bias  for layernorm
config.block_size = 256 # the preset length of seq, 
config.batch_size = 32 # the preset length of seq, 

paras_train = edict()
paras_train.batch_size = 64
paras_train.niter = 500
paras_train.loss_out = 10
paras_train.clip = 1 # 
paras_train.lr_step = 200

In [41]:
def trunc_mse_loss(Y, Y_pred):
    diff = (Y - Y_pred)
    #abs_diff = torch.abs(Y - Y_pred)
    #v = torch.mean(abs_diff[abs_diff < (abs_diff.mean()+1*abs_diff.std())]**2)
    return (diff**2).median()

In [42]:
net = myNet(config)
pos_enc = generate_position_encode(config.block_size, config.nfeature).unsqueeze(0)

loss_fn = nn.MSELoss(reduction="none")
loss_fn = trunc_mse_loss

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=0)
scheduler = ExponentialLR(optimizer, gamma=0.3, verbose=True)

number of parameters: 1.99M
Adjusting learning rate of group 0 to 1.0000e-04.


In [ ]:
dat1 = data[0]
for ix in range(92):
    print(data[ix].std())

423.27496
439.4143
700.7775
625.4858
21543.672
11648.712
34832.87
16833.91
23408.102
12655.108
18435.816
26397.918
26156.062
7215.761
24.011957
14644.457
19681.895
4.353186
10223.101
3.3560998
59921.79
18277.637
18.830772
19764.154
10978.8955
13510.064
350.7893
5.863027
5922.1562
12403.258
31.640512
12133.332
19325.607
37.073875
6079.2764
935.894
393.42087
10218.475
3359.081
573.54126
648.3074
182.9855
694.2041
1024.4565
494.40796
647.30786
497.86935
750.86505
1049.2618
642.30444
452.81525
560.69257
373.29852
476.28067
445.18692
683.4876
5988.1274
232.03542
4189.432
5786.858
6975.1284
7634.328
4637.701
7961.357
184.34195
10747.301
14998.881
11651.636
10691.651
14561.659
13065.513
14040.728
22865.3
16305.5
23018.973
24788.496
12136.742
19260.182
7990.7964
19205.566
3081.7654
17674.361
6906.54
1532.1886
7642.5674
20554.59
31848.992
11692.671
36031.18
14564.424
6881.456
7712.752


In [43]:
# training
loss_cur = 0
losses = []
losses_test = []

t0 = time.time()
for ix in range(paras_train.niter):
    X, Y = get_batch(train_data, 
                      batch_size=config.batch_size,  
                      block_size=config.block_size)
    X = X + pos_enc
    # Zero the gradients
    optimizer.zero_grad()
    
    net_out = net(X)
    Y_pred = out2pred(net_out, X)
    loss = loss_fn(Y, Y_pred)
    
    # Perform backward pass
    loss.backward()
    
    torch.nn.utils.clip_grad_norm_(net.parameters(), paras_train.clip)
    # Perform optimization
    optimizer.step()
    
    loss_cur = loss_cur + loss.item()
    if ix % paras_train.loss_out == (paras_train.loss_out-1):
        losses.append(loss_cur/paras_train.loss_out)
        losses_test.append(evaluate(net))
        print(f"At iter {ix+1}/{paras_train.niter}, "
              f"the losses are {loss_cur/paras_train.loss_out:.5E} (train). "
              f"the losses are {losses_test[-1]:.5E} (test). "
              f"The time used is {delta_time(t0):.3f}s. "
             )
        loss_cur = 0
        t0 = time.time()
    
    #if ix % paras_train.lr_step == (paras_train.lr_step-1):
    #    scheduler.step()

At iter 10/500, the losses are 2.71201E+08 (train). the losses are 3.47161E+08 (test). The time used is 7.969s. 
At iter 20/500, the losses are 2.68217E+08 (train). the losses are 3.20031E+08 (test). The time used is 6.554s. 
At iter 30/500, the losses are 4.43794E+08 (train). the losses are 4.20303E+08 (test). The time used is 8.198s. 
At iter 40/500, the losses are 4.19364E+08 (train). the losses are 6.41405E+08 (test). The time used is 7.147s. 
At iter 50/500, the losses are 1.91561E+08 (train). the losses are 1.05919E+08 (test). The time used is 8.997s. 
At iter 60/500, the losses are 1.06907E+08 (train). the losses are 4.95904E+08 (test). The time used is 9.049s. 
At iter 70/500, the losses are 6.72639E+08 (train). the losses are 1.79605E+07 (test). The time used is 9.060s. 
At iter 80/500, the losses are 2.74807E+08 (train). the losses are 1.85639E+08 (test). The time used is 7.485s. 
At iter 90/500, the losses are 3.41928E+08 (train). the losses are 1.00795E+08 (test). The time 